In [1]:
!pip install speechbrain
!pip install git+https://github.com/huggingface/transformers.git
!pip install --upgrade accelerate


[notice] A new release of pip is available: 24.3.1 -> 25.1
[notice] To update, run: python.exe -m pip install --upgrade pip


  Cloning https://github.com/huggingface/transformers.git to c:\users\tejas\appdata\local\temp\pip-req-build-k1v5b7nd
  Resolved https://github.com/huggingface/transformers.git to commit fe29b8c487dfe554ee44e2a359605f82651e9095
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'


  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git 'C:\Users\tejas\AppData\Local\Temp\pip-req-build-k1v5b7nd'

[notice] A new release of pip is available: 24.3.1 -> 25.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.3.1 -> 25.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [29]:
from transformers import SpeechT5Processor, SpeechT5ForTextToSpeech, SpeechT5HifiGan
from datasets import load_dataset
import soundfile as sf
import torch

# Config
class Config:
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    sampling_rate = 16000

# Load models with safetensors (if available)
processor = SpeechT5Processor.from_pretrained("microsoft/speecht5_tts")
model = SpeechT5ForTextToSpeech.from_pretrained("microsoft/speecht5_tts", use_safetensors=True).to(Config.device)
vocoder = SpeechT5HifiGan.from_pretrained("microsoft/speecht5_hifigan", use_safetensors=True)

# Load speaker embeddings
embeddings_dataset = load_dataset("Matthijs/cmu-arctic-xvectors", split="validation")
speaker_embeddings = torch.tensor(embeddings_dataset[0]["xvector"]).unsqueeze(0)

# Inference function
def infer(text, emotion, file_name):
    combined_input = f"{text} [EMOTION] {emotion}"
    inputs = processor(text=combined_input, return_tensors="pt").to(Config.device)
    speech = model.generate_speech(inputs["input_ids"], speaker_embeddings.to(Config.device), vocoder=vocoder)
    sf.write(f"{file_name}.wav", speech.cpu().numpy(), samplerate=Config.sampling_rate)

config.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/50.6M [00:00<?, ?B/s]

README.md:   0%|          | 0.00/1.01k [00:00<?, ?B/s]

cmu-arctic-xvectors.py:   0%|          | 0.00/1.36k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


0000.parquet:   0%|          | 0.00/21.3M [00:00<?, ?B/s]

Generating validation split:   0%|          | 0/7931 [00:00<?, ? examples/s]

In [ ]:
from transformers import SpeechT5Processor
from transformers import SpeechT5ForTextToSpeech
from transformers import SpeechT5HifiGan
from datasets import load_dataset,Audio
import soundfile as sf

import os
import torch
import torchaudio
import datasets
import pandas as pd
from torch import manual_seed
from datasets import Dataset

from speechbrain.pretrained import EncoderClassifier
import warnings
warnings.simplefilter("ignore")

c:\Users\tejas\AppData\Local\Programs\Python\Python310\lib\inspect.py:869: UserWarning: Module 'speechbrain.pretrained' was deprecated, redirecting to 'speechbrain.inference'. Please update your script. This is a change from SpeechBrain 1.0. See: https://github.com/speechbrain/speechbrain/releases/tag/v1.0.0
  if ismodule(module) and hasattr(module, '__file__'):
C:\Users\tejas\AppData\Local\Temp\ipykernel_16784\3761771796.py:15: UserWarning: Module 'speechbrain.pretrained' was deprecated, redirecting to 'speechbrain.inference'. Please update your script. This is a change from SpeechBrain 1.0. See: https://github.com/speechbrain/speechbrain/releases/tag/v1.0.0
  from speechbrain.pretrained import EncoderClassifier


In [3]:
# !pip install --upgrade torch
# !pip install torch==2.1.0

In [4]:
# import torch; print(torch.__version__)

In [5]:
manual_seed(32)
class Config:
    device='cuda'if torch.cuda.is_available() else 'cpu'
    sampling_rate=16000

In [58]:
# processor = SpeechT5Processor.from_pretrained("microsoft/speecht5_tts")
# model = SpeechT5ForTextToSpeech.from_pretrained("microsoft/speecht5_tts")
# model=model.to(Config.device)
# vocoder = SpeechT5HifiGan.from_pretrained("microsoft/speecht5_hifigan") 
# embeddings_dataset = load_dataset("Matthijs/cmu-arctic-xvectors", split="validation")
# speaker_embeddings = torch.tensor(embeddings_dataset[0]["xvector"]).unsqueeze(0) 
# def infer(text,emotion,file_name):
#     combined_input = f"{text} [EMOTION] {emotion}"
#     inputs = processor(text=combined_input, return_tensors="pt")
#     speech = model.generate_speech(inputs["input_ids"], speaker_embeddings, vocoder=vocoder)
#     sf.write(f"{file_name}.wav", speech.numpy(), samplerate=16000)

In [28]:
!pip install --upgrade torch torchaudio torchvision

In [27]:
import torch
print(torch.__version__)

2.5.1+cu118


In [31]:
emo_abv={'fru': 'Frustration',
'exc': 'Excitement',
'neu': 'Neutral',
'ang': 'Anger',
'sad': 'Sadness',
'hap': 'Happiness',
'sur': 'Surprise',
'fea': 'Fear',
'oth': 'Other',
'dis': 'Disgust'}

path="./data/iemocapTrans.csv"
df=pd.read_csv(path)
df.drop(['_id','translated'],axis=1,inplace=True)
df['emotion']=df['emotion'].apply(lambda x: emo_abv[x])
df.columns=['activation', 'dominance', 'emotion', 'end_time', 'start_time', 'audiofile_name','text', 'valence']

audio_path="./data/Iemocap_audio/iemocap_audio/IEMOCAP_wav"
df['audio']=df['audiofile_name'].apply(lambda x: torchaudio.load(os.path.join(audio_path,x+".wav"))[0])
df['text']=df['text']+" [emotion] "+df['emotion']

df_=df[['text','audio']]
dataset = Dataset.from_dict({'text':df_['text'],'target':df_['audio'].to_list()})

In [32]:
spk_model_name = "speechbrain/spkrec-xvect-voxceleb"
speaker_model = EncoderClassifier.from_hparams(
    source=spk_model_name, 
    run_opts={"device": Config.device}, 
    savedir=os.path.join("/tmp", spk_model_name)
)

def create_speaker_embedding(waveform):
    with torch.no_grad():
        speaker_embeddings = speaker_model.encode_batch(torch.tensor(waveform))
        speaker_embeddings = torch.nn.functional.normalize(speaker_embeddings, dim=2)
        speaker_embeddings = speaker_embeddings.squeeze().cpu().numpy()
    return speaker_embeddings

def prepare_dataset(single_data):
    example = processor(
        text=single_data["text"],
        audio_target=single_data['target'], 
        sampling_rate=Config.sampling_rate,
        return_attention_mask=False,
    )
    example["labels"] = example["labels"][0]
    example["speaker_embeddings"] = create_speaker_embedding(single_data['target'])
    return example

dataset = dataset.map(prepare_dataset, remove_columns=dataset.column_names)

hyperparams.yaml:   0%|          | 0.00/2.04k [00:00<?, ?B/s]

embedding_model.ckpt:   0%|          | 0.00/16.9M [00:00<?, ?B/s]

mean_var_norm_emb.ckpt:   0%|          | 0.00/3.20k [00:00<?, ?B/s]

classifier.ckpt:   0%|          | 0.00/15.9M [00:00<?, ?B/s]

label_encoder.txt:   0%|          | 0.00/129k [00:00<?, ?B/s]

Map:   0%|          | 0/10039 [00:00<?, ? examples/s]

In [33]:
def is_not_too_long(input_ids):
    input_length = len(input_ids)
    return input_length < 200

dataset = dataset.filter(is_not_too_long, input_columns=["input_ids"])
dataset = dataset.train_test_split(test_size=0.1)

Filter:   0%|          | 0/10039 [00:00<?, ? examples/s]

In [34]:
from dataclasses import dataclass
from typing import Any, Dict, List, Union

@dataclass
class TTSDataCollatorWithPadding:
    processor: Any
        
    def __call__(self, features):
        input_ids = [{"input_ids": feature["input_ids"]} for feature in features]
        label_features = [{"input_values": feature["labels"]} for feature in features]
        speaker_features = [feature["speaker_embeddings"] for feature in features]

        batch = processor.pad(
            input_ids=input_ids,
            labels=label_features,
            return_tensors="pt")        

        batch["labels"] = batch["labels"].masked_fill(batch.decoder_attention_mask.unsqueeze(-1).ne(1), -100)
        del batch["decoder_attention_mask"]

        if model.config.reduction_factor > 1:
            target_lengths = torch.tensor([len(feature["input_values"]) for feature in label_features])
            target_lengths = target_lengths.new([length - length % model.config.reduction_factor for length in target_lengths])
            max_length = max(target_lengths)
            batch["labels"] = batch["labels"][:, :max_length]
        batch["speaker_embeddings"] = torch.tensor(speaker_features)
        
        return batch
    
data_collator = TTSDataCollatorWithPadding(processor=processor)

### Training loop

In [38]:
from transformers import Seq2SeqTrainingArguments
from transformers import Seq2SeqTrainer

a = Seq2SeqTrainingArguments()
training_args = Seq2SeqTrainingArguments(
    output_dir="./out_dir", 
    per_device_train_batch_size=16,
    gradient_accumulation_steps=2,
    learning_rate=1e-5,
    warmup_steps=500,
    max_steps=4000,
    gradient_checkpointing=True,
    eval_strategy="steps",
    per_device_eval_batch_size=8,
    save_steps=1000,
    eval_steps=1000,
    logging_steps=25,
    report_to=["tensorboard"],
    load_best_model_at_end=True,
    greater_is_better=False,
    label_names=["labels"],
    num_train_epochs=1.0)


trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    data_collator=data_collator,
    tokenizer=processor.tokenizer)

trainer.train()

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...


Step,Training Loss,Validation Loss


KeyboardInterrupt: 

### Function to perform Inference

In [ ]:
from IPython.display import Audio

example = dataset["test"][304]
speaker_embeddings = torch.tensor(example["speaker_embeddings"]).unsqueeze(0)
torch.save(speaker_embeddings,'speaker_embeddings.pt')

def gen(prompt,emo,model,speaker_embeddings):
    text=prompt+" [emotion] "+emo
    inputs = processor(text=text, return_tensors="pt")
    spectrogram = model.generate_speech(inputs["input_ids"].to('cpu'), speaker_embeddings.to('cpu'))
    with torch.no_grad():
            speech=vocoder(spectrogram)
    return Audio(speech.cpu().numpy(), rate=16000)

### Generating TTS with emotion

In [ ]:
audio=gen('where do you think you are going from here','Happiness',trainer.model.to('cpu'),torch.load('speaker_embeddings.pt'))
audio

### Ignore below code

In [44]:
# import torch
# from transformers import SpeechT5Processor, SpeechT5ForTextToSpeech, SpeechT5HifiGan
# from IPython.display import Audio

# # Configuration
# class Config:
#     device = 'cuda' if torch.cuda.is_available() else 'cpu'
#     sampling_rate = 16000

# # Load processor and vocoder (these don't change)
# processor = SpeechT5Processor.from_pretrained("microsoft/speecht5_tts")
# vocoder = SpeechT5HifiGan.from_pretrained(
#     "microsoft/speecht5_hifigan", 
#     use_safetensors=True
# )

# # Choose which checkpoint to load (select the highest number for best results)
# checkpoint_path = "./out_dir/checkpoint-4000"  # or whichever checkpoint you prefer

# # Load the fine-tuned model from checkpoint
# model = SpeechT5ForTextToSpeech.from_pretrained(checkpoint_path).to(Config.device)

# # Load speaker embeddings (assuming they're saved in the out_dir)
# speaker_embeddings = torch.load("./out_dir/speaker_embeddings.pt").to(Config.device)

# def generate_speech(prompt, emotion, model, speaker_embeddings):
#     """Generate speech from text using the fine-tuned model"""
#     # Prepare input text with emotion tag
#     text = f"{prompt} [emotion] {emotion}"
    
#     # Process text and generate spectrogram
#     inputs = processor(text=text, return_tensors="pt").to(Config.device)
    
#     with torch.no_grad():
#         # Generate speech
#         spectrogram = model.generate_speech(
#             inputs["input_ids"], 
#             speaker_embeddings
#         )
        
#         # Convert spectrogram to waveform
#         speech = vocoder(spectrogram)
    
#     return speech.cpu().numpy()

# # Example usage
# prompt = "where do you think you are going from here"
# emotion = "Happiness"  # Use one of your emotion labels

# # Generate and play audio
# audio = generate_speech(prompt, emotion, model, speaker_embeddings)
# Audio(audio, rate=Config.sampling_rate)

In [56]:
# import torch
# from transformers import SpeechT5Config, SpeechT5Processor, SpeechT5ForTextToSpeech
# from transformers import SpeechT5HifiGan, SpeechT5HifiGanConfig
# import os
# from safetensors.torch import load_file

# class Config:
#     device = 'cuda' if torch.cuda.is_available() else 'cpu'
#     sampling_rate = 16000

# processor = SpeechT5Processor.from_pretrained("microsoft/speecht5_tts")

# checkpoint_path = "./out_dir/checkpoint-4000"
# model = SpeechT5ForTextToSpeech.from_pretrained(
#     checkpoint_path,
#     local_files_only=True,
#     use_safetensors=True if os.path.exists(os.path.join(checkpoint_path, "model.safetensors")) else False
# ).to(Config.device)

# vocoder_config = SpeechT5HifiGanConfig.from_pretrained("microsoft/speecht5_hifigan")
# vocoder = SpeechT5HifiGan(vocoder_config).to(Config.device)

# try:
#     vocoder_path = "microsoft/speecht5_hifigan"
#     if os.path.exists(vocoder_path):
#         vocoder_state_dict = torch.load(
#             os.path.join(vocoder_path, "pytorch_model.bin"),
#             map_location="cpu",
#             weights_only=True
#         )
#     else: 
#         from huggingface_hub import hf_hub_download
#         vocoder_state_dict = torch.load(
#             hf_hub_download(repo_id="microsoft/speecht5_hifigan", filename="pytorch_model.bin"),
#             map_location="cpu",
#             weights_only=True
#         )
    
#     vocoder.load_state_dict({
#         k: v for k, v in vocoder_state_dict.items()
#         if k in vocoder.state_dict()
#     }, strict=False)
# except Exception as e:
#     print(f"Vocoder loading failed, trying alternative method: {e}")
#     vocoder = SpeechT5HifiGan.from_pretrained(
#         "microsoft/speecht5_hifigan",
#         use_safetensors=True,
#         ignore_mismatched_sizes=True
#     ).to(Config.device)

# speaker_embeddings = torch.load(
#     './out_dir/speaker_embeddings.pt',
#     map_location="cpu",
#     weights_only=True
# ).to(Config.device)

# def generate_speech(prompt, emotion, max_length=500):
#     try:
#         text = f"{prompt} [emotion] {emotion}"
#         inputs = processor(text=text, return_tensors="pt").to(Config.device)
        
#         with torch.no_grad():
#             spectrogram = model.generate_speech(
#                 inputs["input_ids"],
#                 speaker_embeddings,
#                 max_length=max_length
#             )
#             speech = vocoder(spectrogram)
#         return speech.cpu().numpy()
#     except Exception as e:
#         print(f"Generation failed: {e}")
#         return None


In [57]:

# from IPython.display import Audio
# audio = generate_speech("This solution should now work perfectly", "Happiness")
# if audio is not None:
#     display(Audio(audio, rate=Config.sampling_rate))
# else:
#     print("Audio generation failed")